In [16]:
# Ejemplos de endpoints que puedes consultar para obtener diferentes tipos de datos:

#Comics: https://gateway.marvel.com/v1/public/comics
#Characters: https://gateway.marvel.com/v1/public/characters
#Creators: https://gateway.marvel.com/v1/public/creators
#Events: https://gateway.marvel.com/v1/public/events
#Series: https://gateway.marvel.com/v1/public/series
#Stories: https://gateway.marvel.com/v1/public/stories

import requests
import hashlib
import time
from requests.packages.urllib3.contrib import pyopenssl #Para evitar un error con la biblioteca SSL/TLS que me imepedía conectar y forzar a Python a utilizar pyOpenSSL
import pandas as pd


pyopenssl.inject_into_urllib3()


public_key = 'XXXX' # Reemplaza esto con tu clave privada real
private_key = 'YYYY'  # Reemplaza esto con tu clave privada real


timestamp = str(int(time.time()))
to_hash = timestamp + private_key + public_key
hash_value = hashlib.md5(to_hash.encode()).hexdigest()


params = {
    'apikey': public_key,
    'ts': timestamp,
    'hash': hash_value
}


response = requests.get('https://gateway.marvel.com/v1/public/comics', params=params)


comics = response.json()['data']['results']


df = pd.DataFrame(columns=['Title', 'UPC', 'Description', 'URLs', 'Creators', 'Price'])


for i, comic in enumerate(comics):
    title = comic.get('title', None)
    upc = comic.get('upc', None)
    
    textObjects = comic.get('textObjects', [{}])
    description = textObjects[0].get('text', None) if textObjects else None
    if description:
        description = description.replace('"', '')  # Elimina las comillas
    
    urls = comic.get('urls', [])
    url_list = ', '.join([url['url'] for url in urls if 'url' in url])
    
    creators = comic.get('creators', {}).get('items', [])
    creator_names = ', '.join([creator.get('name', '') for creator in creators])
    
    price = comic.get('prices', [{}])[0].get('price', None)
    
    df.loc[i] = [title, upc, description, url_list, creator_names, price]